<a href="https://colab.research.google.com/github/Shkum/IT_STEP/blob/ML/module3/%D0%9F%D1%80%D0%B0%D0%BA%D1%82%D0%B8%D0%BA%D0%B0_%D0%B4%D0%B5%D1%80%D0%B5%D0%B2%D0%B0_%D1%80%D1%96%D1%88%D0%B5%D0%BD%D1%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [160]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

from sklearn.linear_model import LinearRegression

from sklearn import metrics

from sklearn import tree

In [161]:
df = pd.read_csv("https://raw.githubusercontent.com/HalyshAnton/IT-Step-Pyton-AI/main/module3/data/Used%20Car%20Dataset.csv", index_col="Unnamed: 0")
df.drop(columns=['car_name', 'registration_year', 'ownsership', 'transmission'], inplace=True)

Дані про ціни вживаних автомобілів у Індії
* car_name - назва авто у форматі 'рік компанія основна назва'
* registration_year - місяць та рік реєстрації
* ownsership - кількість власників у форматі 'Second owner'
* transmission - тип коробки передач
* insurance_validity - тип страховки
* fuel_type - вид палива
* seats - кількість сидінь
* kms_driven - пробіг
* manufacturing_year - рік виробництва, має тип object, потрібно застосувати astype(int)
* mileage(kmpl) - скільки кілометрів проїде за літр палива
* engine(cc) - об'єм двигуна у мл
* max_power(bhp) - потужність у кінських силах
* torque(Nm) - крутний момент двигуна
* price(in lakhs) - ціна у сто тисяч рупій

[Повний аналіз даних](https://www.kaggle.com/code/abdelrasoul/used-cars-prices-prediction)

In [162]:
df.head()

,insurance_validity,fuel_type,seats,kms_driven,manufacturing_year,mileage(kmpl),engine(cc),max_power(bhp),torque(Nm),price(in lakhs)
0,Comprehensive,Petrol,5,56000,2017,7.81,2996.0,2996.0,333.0,63.75
1,Comprehensive,Petrol,5,30615,2020,17.40,999.0,999.0,9863.0,8.99
2,Comprehensive,Diesel,5,24000,2018,20.68,1995.0,1995.0,188.0,23.75
3,Comprehensive,Petrol,5,18378,2019,16.50,1353.0,1353.0,13808.0,13.56
4,Comprehensive,Petrol,5,44900,2019,14.67,1798.0,1798.0,17746.0,24.00


In [163]:
df = df[['fuel_type', 'kms_driven', 'engine(cc)', 'price(in lakhs)']]
df.head()

,fuel_type,kms_driven,engine(cc),price(in lakhs)
0,Petrol,56000,2996.0,63.75
1,Petrol,30615,999.0,8.99
2,Diesel,24000,1995.0,23.75
3,Petrol,18378,1353.0,13.56
4,Petrol,44900,1798.0,24.00


# Завдання 1
Очистіть дані від викидів(лише `price`), розділіть на тренувальну та тестову чатини

In [164]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1553 entries, 0 to 1552
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   fuel_type        1553 non-null   object 
 1   kms_driven       1553 non-null   int64  
 2   engine(cc)       1550 non-null   float64
 3   price(in lakhs)  1553 non-null   float64
dtypes: float64(2), int64(1), object(1)
memory usage: 60.7+ KB


In [165]:
def remove_outliers(df, column_name):
  column = df[column_name]
  q1 = column.quantile(q=0.25)
  q3 = column.quantile(q=0.75)
  iqr = q3 - q1
  lower = q1 - 1.5*iqr
  upper = q3 + 1.5*iqr
  return df[(column >= lower) & (column <= upper)]

df = remove_outliers(df, 'price(in lakhs)')

In [166]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1362 entries, 1 to 1552
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   fuel_type        1362 non-null   object 
 1   kms_driven       1362 non-null   int64  
 2   engine(cc)       1359 non-null   float64
 3   price(in lakhs)  1362 non-null   float64
dtypes: float64(2), int64(1), object(1)
memory usage: 53.2+ KB


In [167]:
df.shape

(1362, 4)

In [168]:
y = df['price(in lakhs)']
X = df.drop(columns='price(in lakhs)')

In [169]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=42)

# Завдання 2
Створіть Pipeline для обробки даних

In [170]:
num_columns = X.select_dtypes(include="number").columns
cat_columns = X.select_dtypes(include="object").columns

In [171]:
num_transformer = Pipeline(
    steps=[("imputer", SimpleImputer(strategy="median"))]
    )

In [172]:

cat_transformer = Pipeline(
    steps=[("imputer", SimpleImputer(strategy="most_frequent")),
           ("encoder", OneHotEncoder())])

In [173]:

preprocessor = ColumnTransformer(
    transformers=[
        ("num", num_transformer, num_columns),
        ("cat", cat_transformer, cat_columns),
    ]
)

preprocessor

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median'))]),
                                 Index(['kms_driven', 'engine(cc)'], dtype='object')),
                                ('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('encoder', OneHotEncoder())]),
                                 Index(['fuel_type'], dtype='object'))])

In [174]:
preprocessor.fit(X_train)
preprocessor.transform(X_train)

array([[1.6683e+04, 1.9560e+03, 0.0000e+00, 0.0000e+00, 1.0000e+00,
        0.0000e+00],
       [6.2000e+02, 8.1800e+03, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        1.0000e+00],
       [3.5000e+04, 1.5820e+03, 0.0000e+00, 0.0000e+00, 1.0000e+00,
        0.0000e+00],
       ...,
       [1.0400e+04, 9.9900e+02, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        1.0000e+00],
       [8.7109e+04, 8.2000e+01, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        1.0000e+00],
       [3.2000e+04, 1.3950e+03, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        1.0000e+00]])

# Завдання 3
Створіть остаточну модель та натренуйте її

In [175]:
model = Pipeline(
    [("prep", preprocessor),
     ("model", tree.DecisionTreeRegressor(max_depth=4,
                                          random_state=42)
     )
     ]
)

model

Pipeline(steps=[('prep',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median'))]),
                                                  Index(['kms_driven', 'engine(cc)'], dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEncoder())]),
                                                  Index(['fuel_type'], dtype='object'))])),
                ('model', DecisionTreeRegressor(max_depth=4, random_state=42))])

In [176]:
model.fit(X_train, y_train)

Pipeline(steps=[('prep',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median'))]),
                                                  Index(['kms_driven', 'engine(cc)'], dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEncoder())]),
                                                  Index(['fuel_type'], dtype='object'))])),
                ('model', DecisionTreeRegressor(max_depth=4, random_state=42))])

# Завдання 4
Виведіть основні метрики моделі

In [177]:
def get_metrics(y_true, y_pred, name="model"):
  df = pd.DataFrame()
  df.loc["MAE", name] = metrics.mean_absolute_error(y_true, y_pred)
  #df.loc["MSE", name] = metrics.mean_squared_error(y_true, y_pred)
  df.loc["RMSE", name] = metrics.mean_squared_error(y_true, y_pred) ** 0.5
  df.loc["R2", name] = metrics.r2_score(y_true, y_pred)
  return df.round(2)

In [178]:
y_pred = model.predict(X_test)

In [179]:
all_metrics = pd.DataFrame()

all_metrics["train"] = get_metrics(y_train, model.predict(X_train))
all_metrics["test"] = get_metrics(y_test, y_pred)

In [180]:
all_metrics

,train,test
MAE,2.70,3.04
RMSE,4.12,4.72
R2,0.67,0.57


# Завдання 5
Змініть параметри моделі та попробуйте покращити результат

In [181]:
model = Pipeline(
    [("prep", preprocessor),
     ("model", tree.DecisionTreeRegressor(max_depth=10,
                                          random_state=42)
     )
     ]
)

model

Pipeline(steps=[('prep',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median'))]),
                                                  Index(['kms_driven', 'engine(cc)'], dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEncoder())]),
                                                  Index(['fuel_type'], dtype='object'))])),
                ('model',
                 DecisionTreeRegressor(max_depth=10, random_state=42))])

In [182]:
model.fit(X_train, y_train)

Pipeline(steps=[('prep',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median'))]),
                                                  Index(['kms_driven', 'engine(cc)'], dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEncoder())]),
                                                  Index(['fuel_type'], dtype='object'))])),
                ('model',
                 DecisionTreeRegressor(max_depth=10, random_state=42))])

In [183]:
def get_metrics(y_true, y_pred, name="model"):
  df = pd.DataFrame()
  df.loc["MAE", name] = metrics.mean_absolute_error(y_true, y_pred)
  #df.loc["MSE", name] = metrics.mean_squared_error(y_true, y_pred)
  df.loc["RMSE", name] = metrics.mean_squared_error(y_true, y_pred) ** 0.5
  df.loc["R2", name] = metrics.r2_score(y_true, y_pred)
  return df.round(2)

In [184]:
y_pred = model.predict(X_test)

In [185]:
all_metrics = pd.DataFrame()

all_metrics["train"] = get_metrics(y_train, model.predict(X_train))
all_metrics["test"] = get_metrics(y_test, y_pred)

In [186]:
all_metrics

,train,test
MAE,1.16,1.96
RMSE,2.11,3.21
R2,0.91,0.80


# Завдання 6
Збережіть модель

In [186]:
import joblib

In [ ]:
joblib.dump(model, 'my_model')